## Create a short overview notebook for important general information about the data

In [3]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [7]:
df_train =  pd.read_csv("../data/train.csv")
df_train.head(5)


,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750


In [10]:
df_train.describe()
print(df_train.isna().sum())

sample_id        0
image_path       0
Sampling_Date    0
State            0
Species          0
Pre_GSHH_NDVI    0
Height_Ave_cm    0
target_name      0
target           0
dtype: int64


In [12]:
target_names = pd.unique(df_train["target_name"])
print(target_names) #GDM being "Green dry mass"

['Dry_Clover_g' 'Dry_Dead_g' 'Dry_Green_g' 'Dry_Total_g' 'GDM_g']


In [ ]:
# Example manipulations and create new short-id column
plt.figure(figsize=(10,6))
# show difference in lengths (example) - keep for context if needed
print(len(df_train['sample_id'][1]) - len('_Dry_Clover_g'))
# print a single sample's first 12 characters
print(df_train['sample_id'][250][0:12])
# Create a new column 'sample_short' containing the first 12 chars of sample_id
df_train['sample_short'] = df_train['sample_id'].str[:12]
# print first few values to verify
print(df_train['sample_short'].head())

ID1244346858
0     ID1011485656__Dry_Clover_g
1       ID1011485656__Dry_Dead_g
2      ID1011485656__Dry_Green_g
3      ID1011485656__Dry_Total_g
4            ID1011485656__GDM_g
5     ID1012260530__Dry_Clover_g
6       ID1012260530__Dry_Dead_g
7      ID1012260530__Dry_Green_g
8      ID1012260530__Dry_Total_g
9            ID1012260530__GDM_g
10    ID1025234388__Dry_Clover_g
11      ID1025234388__Dry_Dead_g
Name: sample_id, dtype: object
